# Chapter7. $n$-step Bootstrapping

- TD(one-step)
 - 에피소드를 끝까지 보지 않고도 학습 가능 ($\rightarrow$ 온라인 학습 가능)
 - 종료 상태가 없는(continuing, non-terminating) 환경에서도 사용가능
 - bias가 클 수 있고 초기값에 민감하다.
- MC
 - 에피소드를 끝까지 봐야한다.($\rightarrow$ 온라인 학습 불가)
 - 종료 상태가 있는(episodic, terminating) 환경에서만 사용 가능
 - bias가 상대적으로 작고 초기값에 덜 민감하다.
 
> MC와 TD(1)는 위와 같은 장단점을 갖고 있다.    
MC는 다 보고 업데이트하고, TD(1)은 하나식만 보고 업데이트를 한다.    
이는 에피소드를 다루는 방식에 있어서 양 극단이라 할 수 있다.     
전부 혹은 하나가 아니라 적당히 $n$개를 보고 업데이트 하는 방식을 생각해 볼 수 있다.

### 7.1 $n$-step TD Prediction

> **MC Prediction**은 각 에피소드의 마지막 상태까지의 반환값(return, $G_t$)을 타겟으로 $V(S_t)$를 업데이트 한다. 이때 $G_t$는 첫 보상($R_{t+1})$부터 마지막 보상($R_T$)까지 점점 더 할인된 값들의 합이다.

$$\begin{align}
V(S_t) &\leftarrow V(S_t) + \alpha \bigg[ G_t - V(S_t) \bigg] \\
G_t &\doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots | r^{T-t-1} R_T \
\end{align}$$

> 반면 **one-step TD Prediction**혹은 TD(0)는 상태 $S_t$의 보상($R_{t+1}$)과     
$\gamma$만큼 할인된 다음 상태 $S_{t+1}$의 가치 추정치($\gamma V(S_{t+1})$)를 더한 값을 타겟으로           
$v_{\pi}(S_t)$에 대한 추정치 $V(S_t)$를 업데이트 한다.

$$V(S_t) \leftarrow V(S_t) + \alpha \bigg[ R_{t+1} + \gamma V(S_{t+1}) - V(S_t)\bigg]$$



> TD(0)에서 하나의 다음 상태를 보고 업데이트 했다면 **n-step TD Prediction**은 $n$개의 다음 상태를 보고 업데이트 하겠다는 것이다. 예를들어 2-step TD에서는 아래와 같이 $V(S_t)$업데이트 한다. <font color='grey'>$$V(S_t) \leftarrow V(S_t) + \alpha \bigg[ R_{t+1} + \gamma R_{t+1} + \gamma^2 V(S_{t+2}) - V(S_t)\bigg]$$</font>

> 그런데 n-step TD를 표현하기 위해 몇가지 표기법을 도입한다.     
우선 $t$시점의 보상($R_t$)부터 $t+n$ 시점까지의 보상($R_{t+n}$)을 사용하여 추정한 반환값(return)을 $G_{t:t+n}$이라 표기하고,     
$t+n$시점에 추정한 $t$시점의 상태($S_t$)의 가치를 $V_{t+n}(S_t)$라 표기한다.    
($V$의 밑첨자는 가치 평가 시점을 의미하며 $S$의 밑첨자는 어느 상태인지를 나타낸다.)

$$\begin{align}
G_{t:t+n} &\doteq R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n V_{t+n-1}(S_{t+n}) \\
V_{t+n}(S_t) &\doteq V_{t+n-1}(S_t) + \alpha \bigg[ G_{t:t+n} - V_{t+n-1}(S_t)\bigg]
\end{align}$$

![Local image](./images/sutton/figure_7_1.png)

> 1-step TD
$$\begin{align}
G_{t:t+1} &\doteq R_{t+1} + \gamma V_{t}(S_{t+1}) \\
V_{t+1}(S_t) &\doteq V_{t}(S_t) + \alpha \bigg[ G_{t:t+1} - V_{t}(S_t) \bigg]
\end{align}$$

> 2-step TD
$$\begin{align}
G_{t:t+2} &\doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 V_{t+1}(S_{t+2}) \\
V_{t+2}(S_t) &\doteq V_{t+1}(S_t) + \alpha \bigg[ G_{t:t+2} - V_{t+1}(S_t) \bigg]
\end{align}$$

> 3-step TD
$$\begin{align}
G_{t:t+3} &\doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 V_{t+2}(S_{t+2}) \\
V_{t+3}(S_t) &\doteq V_{t+2}(S_t) + \alpha \bigg[ G_{t:t+3} - V_{t+2}(S_t) \bigg]
\end{align}$$

###### Example 7.1

'C 0 D 0 E 1'라는 에피소드가 관찰되었을 때 1-step TD는 아래와 같이 $V(E)$만 업데이트 된다.

> - $t=0$
 - 에피소드 : C 0 D
 - $V_0(A) = V_0(B) = V_0(C) = V_0(D) = V_0(E) = 0.5$
 - $S_0:C$, $~~S_1:D$
 - $R_1 = 0$
 - $G_{0:1} = R_{1} + V_0(S_1:D) = 0 + 0.5$
 - $V_1(S_0:C) = V_0(S_0:C) + \alpha \bigg[ G_{0:1} - V_0(S_1:D) \bigg] = 0.5$


> - $t=1$
 - 에피소드 : C 0 D 0 E
 - $V_1(A) = V_1(B) = V_1(C) = V_1(D) = V_1(E) = 0.5$
 - $S_0:C$, $~~S_1:D$, $~~S_2:E$
 - $R_2 = 0$
 - $G_{1:2} = R_{2} + V_1(S_2:E) = 0 + 0.5$
 - $V_2(S_1:D) = V_1(S_1:D) + \alpha \bigg[ G_{1:2} - V_1(S_1:D) \bigg] = 0.5$


> - $t=2$
 - 에피소드 : C 0 D 0 E 1 T
 - $V_2(A) = V_2(B) = V_2(C) = V_2(D) = V_2(E) = 0.5$, $V_2(T)=0$
 - $S_0:C$, $S_1:D$, $S_2:E$, $S_3:T$
 - $R_3=1$
 - $G_{2:3} = R_{3} + V_2(S_3:T) = 1 + 0$
 - $V_3(S_2:E) = V_2(S_2:E) + \alpha \bigg[ G_{2:3} - V_2(S_:E) \bigg] = 0.5 + \bigg[ 1 - \alpha \cdot 0.5 \bigg]$


반면 2-step TD를 사용하면 $V(D)$와 $V(E)$가 업데이트 된다.

> - $t=0$
 - 에피소드 : C 0 D
 - $V_0(A) = V_0(B) = V_0(C) = V_0(D) = V_0(E) = 0.5$
 - $S_0:C$, $~~S_1:D$
 - $R_1 = 0$
 - $\text{No update}$


> - $t=1$
 - 에피소드 : C 0 D 0 E
 - $V_1(A) = V_1(B) = V_1(C) = V_1(D) = V_1(E) = 0.5$
 - $S_0:C$, $~~S_1:D$, $~~S_2:E$
 - $R_1 = 0$, $~~R_2 = 0$
 - $G_{0:2} = R_{1} + R_{2} + V_1(S_2:E) = 0 + 0 + 0.5$
 - $V_2(S_0:C) = V_1(S_0:C) + \alpha \bigg[ G_{0:2} - V_1(S_0:C) \bigg] = 0.5$


> - $t=2$
 - 에피소드 : C 0 D 0 E 1 T
 - $V_2(A) = V_2(B) = V_2(C) = V_2(D) = V_2(E) = 0.5$, $V_2(T)=0$
 - $S_0:C$, $S_1:D$, $S_2:E$, $S_3:T$
 - $R_1 = 0$, $~~R_2 = 0$, $~~R_3=1$
 - $G_{1:3} = R_{2} + R_{3} + V_2(S_3:T) = 0 + 1 + 0$
 - $V_3(S_1:D) = V_2(S_1:D) + \alpha \bigg[ G_{1:3} - V_2(S_1:D) \bigg] = 0.5 + \bigg[ 1 - \alpha \cdot 0.5 \bigg]$
 - $G_{2:4} = R_{3} = 1$
 - $V_3(S_2:E) = V_2(S_2:E) + \alpha \bigg[ G_{2:4} - V_2(S_2:E) \bigg] = 0.5 + \bigg[ 1 - \alpha \cdot 0.5 \bigg]$


또한 3-step TD를 사용하면 $V(C)$, $V(D)$, $V(E)$가 업데이트 된다.

> - $t=0$
 - 에피소드 : C 0 D
 - $V_0(A) = V_0(B) = V_0(C) = V_0(D) = V_0(E) = 0.5$
 - $S_0:C$, $~~S_1:D$
 - $R_1 = 0$
 - $\text{No update}$


> - $t=1$
 - 에피소드 : C 0 D 0 E
 - $V_1(A) = V_1(B) = V_1(C) = V_1(D) = V_1(E) = 0.5$
 - $S_0:C$, $~~S_1:D$, $~~S_2:E$
 - $R_1 = 0$, $~~R_2 = 0$
 - $\text{No update}$

> - $t=2$
 - 에피소드 : C 0 D 0 E 1 T
 - $V_2(A) = V_2(B) = V_2(C) = V_2(D) = V_2(E) = 0.5$, $V_2(T)=0$
 - $S_0:C$, $S_1:D$, $S_2:E$, $S_3:T$
 - $R_1 = 0$, $~~R_2 = 0$, $~~R_3=1$
 - $G_{0:3} = R_1 + R_{2} + R_{3} + V_2(S_3:T) = 0 + 1 + 0$
 - $V_3(S_0:C) = V_2(S_0:C) + \alpha \bigg[ G_{0:3} - V_2(S_0:C) \bigg] = 0.5 + \bigg[ 1 - \alpha \cdot 0.5 \bigg]$
 - $G_{1:4} = R_{2} + R_{3} + 0 + 0 = 1$
 - $V_3(S_1:D) = V_2(S_1:D) + \alpha \bigg[ G_{1:3} - V_2(S_1:D) \bigg] = 0.5 + \bigg[ 1 - \alpha \cdot 0.5 \bigg]$
 - $G_{2:5} = R_{3} = 0 + 1 + 0 + 0 = 1$
 - $V_3(S_2:E) = V_2(S_2:E) + \alpha \bigg[ G_{2:4} - V_2(S_2:E) \bigg] = 0.5 + \bigg[ 1 - \alpha \cdot 0.5 \bigg]$


즉 $n$ 스텝까지 에피소드를 본 후 $n$ 스텝 이전의 상태를 업데이트 한다.

![Local image](./images/sutton/example_6_2.png)

위 random walk는 상태가 5개 인데, 상태가 19개인 경우로 확장하여 서로 다른 $n$과 $\alpha$에 따른 RMS(Root mean square) error를 살펴보면 아래 그래프와 같다. 즉 $\alpha$를 잘 조절한다는 가정하에 좌측의 $n$이 큰 경우는 MC라 생각할 수 있고 우측의 $n=1$인 경우는 TD(0)라고 볼 수 있어서, 적절한 $n$을 선택한 경우 최적의 성능을 낼 수 있다는 것이다.

![Local image](./images/sutton/figure_7_2.png)

### 7.2 $n$-step Sarsa

Sarsa는 On-policy TD Control을 말한다.

Sarsa는 뭐였지?

prediction -> control

state value -> action value

greedy -> epsilon greedy

then n-step Sarsa

### 7.3 $n$-step Off-policy Learning by Importence sampling

.

### 7.4 \*Per-decision Off-policy Methods with Control Variates

.

### 7.5 Off-policy Learning Without Importance Sampling: The $n$-step Tree Backup Algorithm

.

### 7.6 \*A Unifying Algorithm: n-step $Q(\sigma)$

.

.

.

.

.